### 1. 데이터 로드 및 전처리

In [1]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [2]:
df=pd.read_csv('./data/Multiclass_data.csv')
df.head()

,Class,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60
0,HBRG,4.486201,7.522693,0,19.265581,0.0,0.0,0,0.0,4.37144,...,2.247654,1.272134,0.0,0,2.836458,0.0,4.834460,0,1.642702,0
1,HBRG,0.000000,7.836708,0,18.327251,0.0,0.0,0,0.0,0.00000,...,0.000000,0.000000,0.0,0,0.000000,0.0,5.025198,0,0.000000,0
2,HBRG,0.000000,7.439930,0,21.020445,0.0,0.0,0,0.0,0.00000,...,0.000000,0.000000,0.0,0,2.424664,0.0,5.706630,0,0.000000,0
3,HBRG,0.000000,18.827015,0,15.553238,0.0,0.0,0,0.0,0.00000,...,0.000000,1.934435,0.0,0,3.018529,0.0,5.741382,0,0.000000,0
4,HBRG,0.188557,17.855667,0,16.108568,0.0,0.0,0,0.0,0.00000,...,0.000000,1.704666,0.0,0,2.579150,0.0,3.234617,0,0.000000,0


In [3]:
df.isnull().sum()

Class    0
V1       0
V2       0
V3       0
V4       0
        ..
V56      0
V57      0
V58      0
V59      0
V60      0
Length: 61, dtype: int64

In [4]:
X=df.drop(["Class"],axis=1)
Y=df["Class"]
Y.value_counts()

SPOT        1140
RING         824
LSAC         268
BRG          137
L_MKBG       135
HBRG         123
SFPT         100
SORA          73
GUDEONGI      12
Name: Class, dtype: int64

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=120,stratify=Y)
print(Y_train.value_counts())
#print(Y_test.value_counts())

SPOT        912
RING        659
LSAC        214
BRG         110
L_MKBG      108
HBRG         98
SFPT         80
SORA         58
GUDEONGI     10
Name: Class, dtype: int64


### 2 . Under Over sampling 적용 및 결과 확인

In [6]:
from imblearn.under_sampling import *
from imblearn.over_sampling import *
from imblearn.combine import *
from sklearn.ensemble import RandomForestClassifier
from imblearn.combine import SMOTETomek

In [7]:
# 단순 비교
# 주의 사항: TEST 데이터에 오버샘플링 하지 말것.
X_TomekLinks,Y_TomekLinks = TomekLinks().fit_resample(X_train, Y_train)
print("====TomekLinks====")
print(Y_TomekLinks.value_counts())

X_SMOTE, Y_SMOTE = SMOTE(random_state=10).fit_resample(X_train, Y_train)
print("====SMOTE====")
print(Y_SMOTE.value_counts())

X_SMOTETomek, Y_SMOTETomek = SMOTETomek(random_state=10).fit_resample(X_train, Y_train)
print("====SMOTETomek====")
print(Y_SMOTETomek.value_counts())



====TomekLinks====
SPOT        906
RING        645
LSAC        206
BRG         110
L_MKBG      107
HBRG         98
SFPT         80
SORA         58
GUDEONGI     10
Name: Class, dtype: int64
====SMOTE====
LSAC        912
RING        912
SPOT        912
SORA        912
L_MKBG      912
SFPT        912
HBRG        912
BRG         912
GUDEONGI    912
Name: Class, dtype: int64
====SMOTETomek====
LSAC        912
SORA        912
L_MKBG      912
SFPT        912
HBRG        912
BRG         912
GUDEONGI    912
RING        909
SPOT        909
Name: Class, dtype: int64


In [8]:
tree_model1=RandomForestClassifier(random_state=40, max_depth=10)
tree_model2=RandomForestClassifier(random_state=40, max_depth=10)
tree_model3=RandomForestClassifier(random_state=40, max_depth=10)
tree_model4=RandomForestClassifier(random_state=40, max_depth=10)


#기본
tree_model1.fit(X_train, Y_train)

# 언더 샘플링
tree_model2.fit(X_TomekLinks,Y_TomekLinks)

# 오버 샘플링
tree_model3.fit(X_SMOTE,Y_SMOTE)
tree_model4.fit(X_SMOTETomek, Y_SMOTETomek)



pred=tree_model1.predict(X_test)
TomekLinks_pred=tree_model2.predict(X_test)
smote_pred=tree_model3.predict(X_test)
smotetomek_pred=tree_model4.predict(X_test)


In [9]:
print("===================일반=========================")
print("Precision score : {:.4f}".format(precision_score(Y_test,pred,average ='macro')))
print("Recall score : {:.4f}".format(recall_score(Y_test,pred,average ='macro')))
print("F1 score : {:.4f}".format(f1_score(Y_test,pred,average ='macro')))

print("===================TomekLinks=========================")
print("Precision score : {:.4f}".format(precision_score(Y_test,TomekLinks_pred,average ='macro')))
print("Recall score : {:.4f}".format(recall_score(Y_test,TomekLinks_pred,average ='macro')))
print("F1 score : {:.4f}".format(f1_score(Y_test,TomekLinks_pred,average ='macro')))

print("===================SMOTE=========================")
print("Precision score : {:.4f}".format(precision_score(Y_test,smote_pred,average ='macro')))
print("Recall score : {:.4f}".format(recall_score(Y_test,smote_pred,average ='macro')))
print("F1 score : {:.4f}".format(f1_score(Y_test,smote_pred,average ='macro')))

print("===================SMOTETomek=========================")
print("Precision score : {:.4f}".format(precision_score(Y_test,smotetomek_pred,average ='macro')))
print("Recall score : {:.4f}".format(recall_score(Y_test,smotetomek_pred,average ='macro')))
print("F1 score : {:.4f}".format(f1_score(Y_test,smotetomek_pred,average ='macro')))


===================일반=========================
Precision score : 0.8390
Recall score : 0.7814
F1 score : 0.8041
===================TomekLinks=========================
Precision score : 0.8508
Recall score : 0.7921
F1 score : 0.8158
===================SMOTE=========================
Precision score : 0.9521
Recall score : 0.9552
F1 score : 0.9512
===================SMOTETomek=========================
Precision score : 0.9498
Recall score : 0.9516
F1 score : 0.9470


C:\Users\SIL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SIL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
